In [1]:
!pip install --use-deprecated=legacy-resolver tflite-model-maker
!pip install pycocotools
# !pip install -q opencv-python-headless==4.1.2.30

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple
Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [1]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import cv2


import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [2]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import cv2


import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [3]:
labels = {1:'apple', 2:'banana', 3:'orange'}

train_imgs_path = 'images/train'
train_anno_path = 'annotations/train'

test_imgs_path = 'images/test'
test_anno_path = 'annotations/test'

valid_imgs_path = 'images/test'
valid_anno_path = 'annotations/test'

train_data = object_detector.DataLoader.from_pascal_voc(images_dir=train_imgs_path, annotations_dir=train_anno_path, label_map=labels)
test_data = object_detector.DataLoader.from_pascal_voc(images_dir=test_imgs_path, annotations_dir=test_anno_path, label_map=labels)
# valid_data = object_detector.DataLoader.from_pascal_voc(images_dir=valid_imgs_path, annotations_dir=valid_anno_path, label_map=labels)

In [4]:
spec = model_spec.get('efficientdet_lite0')
spec.uri = 'https://storage.googleapis.com/tfhub-modules/tensorflow/efficientdet/lite0/feature-vector/1.tar.gz'
spec.input_image_shape = [512, 512]

In [5]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, )
model.summary()

Epoch 1/50
30/30 [==============================] - 34s 302ms/step - det_loss: 1.6744 - cls_loss: 1.0912 - box_loss: 0.0117 - reg_l2_loss: 0.0632 - loss: 1.7375 - learning_rate: 0.0090 - gradient_norm: 1.5348
Epoch 2/50
30/30 [==============================] - 9s 311ms/step - det_loss: 1.2536 - cls_loss: 0.8332 - box_loss: 0.0084 - reg_l2_loss: 0.0632 - loss: 1.3168 - learning_rate: 0.0100 - gradient_norm: 2.4784
Epoch 3/50
30/30 [==============================] - 9s 301ms/step - det_loss: 0.9447 - cls_loss: 0.6029 - box_loss: 0.0068 - reg_l2_loss: 0.0633 - loss: 1.0080 - learning_rate: 0.0099 - gradient_norm: 2.7833
Epoch 4/50
30/30 [==============================] - 9s 307ms/step - det_loss: 0.8102 - cls_loss: 0.5125 - box_loss: 0.0060 - reg_l2_loss: 0.0633 - loss: 0.8736 - learning_rate: 0.0099 - gradient_norm: 3.0022
Epoch 5/50
30/30 [==============================] - 9s 300ms/step - det_loss: 0.7175 - cls_loss: 0.4391 - box_loss: 0.0056 - reg_l2_loss: 0.0634 - loss: 0.7809 - learn

In [6]:
model.evaluate(test_data)

1/1 [==============================] - 9s 9s/step



{'AP': 0.6633582,
 'AP50': 0.9380468,
 'AP75': 0.78553146,
 'APs': -1.0,
 'APm': 0.63286185,
 'APl': 0.6690955,
 'ARmax1': 0.44833332,
 'ARmax10': 0.7343651,
 'ARmax100': 0.74722224,
 'ARs': -1.0,
 'ARm': 0.725,
 'ARl': 0.7502767,
 'AP_/apple': 0.73577553,
 'AP_/banana': 0.5365916,
 'AP_/orange': 0.71770746}

In [7]:
config = QuantizationConfig.for_float16()
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)

In [8]:
model.evaluate_tflite('model_fp16.tflite', test_data)

60/60 [==============================] - 3s 52ms/step



{'AP': 0.64646935,
 'AP50': 0.9445973,
 'AP75': 0.7496816,
 'APs': -1.0,
 'APm': 0.6108911,
 'APl': 0.6527287,
 'ARmax1': 0.4475,
 'ARmax10': 0.69968253,
 'ARmax100': 0.70047617,
 'ARs': -1.0,
 'ARm': 0.65,
 'ARl': 0.7047679,
 'AP_/apple': 0.7313379,
 'AP_/banana': 0.49570423,
 'AP_/orange': 0.71236587}